In [1]:
!pip install --quiet gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.1 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/phthinh291/DACNTT

Cloning into 'DACNTT'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 231 (delta 38), reused 66 (delta 28), pack-reused 151
Receiving objects: 100% (231/231), 274.40 MiB | 15.99 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [3]:
%cd DACNTT/trained_models

/content/DACNTT/trained_models


In [4]:
# Import trained model from Github

!cp /content/DACNTT/trained_models/* /content/

In [5]:
import pickle
import pandas as pd
import gradio as gr
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

In [6]:
# Load trained models

df_names = [
    "USD_VND_SVR_model", "USD_VND_KNN_model", "USD_VND_RF_model", "USD_VND_XGB_model", "USD_VND_MLP_model",
    "USD_EUR_SVR_model", "USD_EUR_KNN_model", "USD_EUR_RF_model", "USD_EUR_XGB_model", "USD_EUR_MLP_model",
    "EUR_VND_SVR_model", "EUR_VND_KNN_model", "EUR_VND_RF_model", "EUR_VND_XGB_model", "EUR_VND_MLP_model"]

dfs = {name: pd.DataFrame() for name in df_names}

models = [
    "SVR_USD_VND.pkl", "KNN_USD_VND.pkl", "RF_USD_VND.pkl", "XGB_USD_VND.pkl", "MLP_USD_VND.pkl",
    "SVR_USD_EUR.pkl", "KNN_USD_EUR.pkl", "RF_USD_EUR.pkl", "XGB_USD_EUR.pkl", "MLP_USD_EUR.pkl",
    "SVR_EUR_VND.pkl", "KNN_EUR_VND.pkl", "RF_EUR_VND.pkl", "XGB_EUR_VND.pkl", "MLP_EUR_VND.pkl"]

for model, df in zip(models, df_names):
  with open(model, "rb") as f:
    dfs[df] = pickle.load(f)

In [7]:
# Load scale information

with open('scale_info.pkl', 'rb') as file:
  loaded_scale_info = pickle.load(file)

USD_VND_loaded_mean = loaded_scale_info['usd-vnd']['mean']
USD_VND_loaded_scale = loaded_scale_info['usd-vnd']['scale']

USD_EUR_loaded_mean = loaded_scale_info['usd-eur']['mean']
USD_EUR_loaded_scale = loaded_scale_info['usd-eur']['scale']

EUR_VND_loaded_mean = loaded_scale_info['eur-vnd']['mean']
EUR_VND_loaded_scale = loaded_scale_info['eur-vnd']['scale']

In [8]:
# Categorize models by currency pair into separate lists

USD_VND_models = []
for name in df_names[0:5]:
  USD_VND_models.append(dfs[name])

USD_EUR_models = []
for name in df_names[5:10]:
  USD_EUR_models.append(dfs[name])

EUR_VND_models = []
for name in df_names[10:15]:
  EUR_VND_models.append(dfs[name])

In [9]:
# Setup web theme

custom_theme = gr.themes.Base(
    primary_hue="emerald",
    secondary_hue="teal",
    neutral_hue="green",
).set(
    prose_text_size='*text_xxl',
    prose_text_weight='600',
    prose_header_text_weight='700'
)

In [41]:
# Demo web code using Gradio

files_limit = False
features = ['Price', 'Open', 'High', 'Low']
result_df_list = []

def update_input_method(choice):
  global num_of_files
  global valid_df
  global exchange_pair
  global file_index
  global execute_index
  file_index = 0
  execute_index = -5
  num_of_files = 0
  valid_df = []
  exchange_pair = []

  if choice == 'Upload input file(s)':
    return gr.update(visible=True), gr.update(visible=False), gr.update(interactive=True), \
           gr.Row(visible=True), gr.Row(visible=False)
  elif choice == 'Manual Input':
    return gr.update(visible=False), gr.update(visible=True), gr.update(interactive=True), \
           gr.Row(visible=False), gr.Row(visible=True)

def read_files(files):
  global valid_df
  global file_names
  global num_of_files
  global exchange_pair
  global files_limit
  global file_index
  num_of_files = 0
  file_index = 0

  files_limit = False
  valid_df = []
  file_names = []
  exchange_pair = []

  for file in files:
    df = pd.read_csv(file.name, encoding='utf-8').drop(columns=['Vol.','Change %'])
    missing_features = [feature for feature in features if feature not in df.columns]

    if missing_features:
      gr.Warning('There is a file that does not have enough required features')
      files_limit = True

    else:
      if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y').dt.strftime('%d/%m/%Y')

      files_limit = False
      if "USD_VND" in file.name:
        exchange_pair.append("usd-vnd")
      elif "USD_EUR" in file.name:
        exchange_pair.append("usd-eur")
      elif "EUR_VND" in file.name:
        exchange_pair.append("eur-vnd")
      else:
        gr.Warning('Must contain exchange pair information in each file\'s name ' +
                  'with the format XXX_YYY\nExample: USD_VND_data.csv')
        files_limit = True

      num_of_files += 1
      valid_df.append(df)
      file_names.append(file.name)

  if int(num_of_files) > 3:
      gr.Warning('Number of files exceeded the limit.\n Maximum capacity is 3 files')
      files_limit = True

  return None

def check_manual_legit(*arg):
  global num_of_files
  global files_limit
  files_limit = False

  input_df_list = list(arg[:3])
  input_pair_list = list(arg[3:])

  for df, pair in zip(input_df_list, input_pair_list):
    df.replace('', np.nan, inplace=True)
    df = df.dropna(how='all', axis=0)

    if df.empty is True:
      continue

    has_missing_values = df.isna().any().any()
    all_numeric = (df.applymap(lambda x: str(x).isdigit())).all().all()

    if has_missing_values:
      gr.Warning("Please fill all feature values of your inputted rows!")
      return gr.update(value='Invalid format! Please Check and Retry', interactive=True), gr.update(interactive=False)
    else:
      has_alphabetic = (df.applymap(lambda x: str(x).isalpha())).any().any()

      if has_alphabetic:
        gr.Warning("Please fill only with numeric characters!")
        return gr.update(value='Invalid format! Please Check and Retry', interactive=True), gr.update(interactive=False)
      else:
        df = df.replace(',', '', regex=True)
        valid_df.append(df.astype(float))
        exchange_pair.append(pair.lower())
        num_of_files += 1

  return gr.update(value='Valid format!', interactive=False), gr.update(interactive=True)

def check_no_file():
  if num_of_files == 0:
    files_limit = True
    gr.Warning('There is no file to execute')

  return None

def load_df(*arg, prg=gr.Progress(track_tqdm=True)):
  if files_limit is True:
    return [gr.Dataframe(visible=False)] * 3 + [gr.update(interactive=False)]

  display_list = []
  df_list = []
  label_list = []
  num = 0
  prg(0, desc="Starting")
  time.sleep(0.75)

  for df in prg.tqdm(valid_df, desc='Reading data'):

    if exchange_pair[num] == 'usd-vnd':
      display_list.append(gr.update(interactive = False, visible = True,
                                    value=df, label="USD-VND Exchange Rate"))

    elif exchange_pair[num] == 'usd-eur':
      display_list.append(gr.update(interactive = False, visible = True,
                                    value=df, label="USD-EUR Exchange Rate"))

    elif exchange_pair[num] == 'eur-vnd':
      display_list.append(gr.update(interactive = False, visible = True,
                                    value=df, label="EUR-VND Exchange Rate"))

    num += 1
    time.sleep(0.25)

  while len(display_list) < 3:
    display_list.append(gr.update(value=None, visible = False))
    time.sleep(0.25)

  prg(1.0, desc="Preparing to display data")

  return display_list[0], display_list[1], display_list[2], gr.update(interactive=True)

def update_display_visible(*arg):
  n = len(arg)

  if arg[0] == 'initial display':
    return [gr.update(visible=True)] * num_of_files + \
           [gr.update(visible=False)] * (3-num_of_files)

  if arg[0] == 'result dfs':
    return [gr.update(value=None,visible=True)] * (n-1)

  else:
    return [gr.update(visible=True)] * n

def update_display_invisible(*arg):
  n = len(arg)

  return [gr.update(visible=False)] * n

def update_layout_visible(text):
  n = int(text)

  if files_limit is True:
    return [gr.Row(visible=False)] * n + [gr.update(visible=False)]

  else:
    if file_index < num_of_files:
      return [gr.Row(visible=True)] * n + [gr.update(visible=False)]

    else:
      return [gr.Row(visible=False)] * n + [gr.update(visible=False)]

def update_layout_invisible(text):
  n = int(text)

  return [gr.Row(visible=False)] * n

def update_interactive_true(*arg):
  if files_limit is True:
    return gr.update(interactive=False)

  else:
    return gr.update(interactive=True)

def update_interactive_false():
  return gr.update(interactive=False)

def update_btn(input_text):
  text = input_text[:-1]
  state = input_text[-1]

  if files_limit is True:
    return gr.update(value='Execute Prediction', interactive=False)

  if state == '1':
    return gr.update(value=text, interactive=True)
  else:
    return gr.update(value=text, interactive=False)

def update_clear_df(n):
  n = int(n)
  default = pd.DataFrame({'1':[], '2':[], '3':[]}).style.format(precision=6, thousands=',', decimal='.')

  if n == 3:
    return [gr.update(value=default, visible=True)] * 3
  else:
    return [gr.update(value=default)] * n

def cast_to_float(df):
  for col in features:
    if df[col].dtype != 'float64':
      df[col] = df[col].str.replace(',', '').astype(float)

  return df

def update_markdowns():
  if files_limit is True:
    return gr.update(value="None", visible=False)

  if file_index < num_of_files:
    return gr.update(value="Predictions for inputted " + exchange_pair[file_index].upper() + " Exchange Rates",
                     visible=True)
  else:
    return gr.update(value="None", visible=False)

def normalize_value(df, exchange_pair):
  if exchange_pair == 'usd-vnd':
    df[features] = (df[features] - USD_VND_loaded_mean) / USD_VND_loaded_scale

    return df, [USD_VND_loaded_mean[0], USD_VND_loaded_scale[0]]

  elif exchange_pair == 'usd-eur':
    df[features] = (df[features] - USD_EUR_loaded_mean) / USD_EUR_loaded_scale

    return df, [USD_EUR_loaded_mean[0], USD_EUR_loaded_scale[0]]

  elif exchange_pair == 'eur-vnd':
    df[features] = (df[features] - EUR_VND_loaded_mean) / EUR_VND_loaded_scale

    return df, [EUR_VND_loaded_mean[0], EUR_VND_loaded_scale[0]]


def invert_value(value, info):
  mean = info[0]
  scale = info[1]
  value = (value * scale) + mean

  return value

def execute_prediction(prg = gr.Progress(track_tqdm=True)):
  global result_df_list
  global execute_index
  global file_index
  execute_index += 5
  result_df_list = []

  if files_limit is True:
    return [gr.update(value=None)] * 5

  if file_index < num_of_files:
    base_df = valid_df[file_index]
    exchange_type = exchange_pair[file_index]
    input_data = base_df[features]
    input_data = cast_to_float(input_data)
    temp_result = generate_prediction(input_data, exchange_type, prg)
    output_df_list, df_invert_info = temp_result[:5], temp_result[5]

    for df in prg.tqdm(output_df_list, desc='Assigning predicted data'):
      time.sleep(0.25)
      inverted_output = invert_value(df, df_invert_info)
      temp_df = valid_df[file_index].copy()
      temp_df['Output'] = inverted_output
      inverted_df = pd.DataFrame(temp_df).style.format(precision=6, thousands=',', decimal='.')
      result_df_list.append(inverted_df)

    file_index += 1

    prg(0.98, desc='Finalizing and Rendering (this could take up to 60 seconds)')

    return  gr.update(value = result_df_list[0]), gr.update(value = result_df_list[1]), \
              gr.update(value = result_df_list[2]), gr.update(value = result_df_list[3]), \
              gr.update(value = result_df_list[4])

  else:
    return [gr.update(value = None)] * 5

def generate_prediction(input_df, exchange_pair, prg=gr.Progress(track_tqdm=True)):
  prediction_list = []

  if exchange_pair == "usd-vnd":
    normalized_input, normalized_info = normalize_value(input_df, exchange_pair)

    for model in prg.tqdm(USD_VND_models, desc="Generating Prediction"):
      prediction = model.predict(normalized_input)
      prediction_list.append(prediction)

  elif exchange_pair == "usd-eur":
    normalized_input, normalized_info = normalize_value(input_df, exchange_pair)

    for model in prg.tqdm(USD_EUR_models, desc="Generating Prediction"):
      prediction = model.predict(normalized_input)
      prediction_list.append(prediction)

  elif exchange_pair == "eur-vnd":
    normalized_input, normalized_info = normalize_value(input_df, exchange_pair)

    for model in prg.tqdm(EUR_VND_models, desc="Generating Prediction"):
      prediction = model.predict(normalized_input)
      prediction_list.append(prediction)

  return [prediction_list[0], prediction_list[1], prediction_list[2],
          prediction_list[3], prediction_list[4], normalized_info]

with gr.Blocks(theme=custom_theme) as demo:
  gr.Markdown('**CURRENCIES EXCHANGE RATE PREDICTION**')
  gr.Markdown('_Developed by Le Phuoc Thinh_')
  dummy = gr.Textbox(visible=False)
  value_1 = gr.Textbox('1', visible=False)
  value_3 = gr.Textbox('3', visible=False)

  with gr.Row() as radio_row:
    radio = gr.Radio(['Upload input file(s)', 'Manual Input'], label = 'Choose your input type')

  with gr.Row(visible=False) as upload_note_row:
    note = '**INPUT NOTES:**\nThe name of each upload file needs to contain one of the following:\n' + \
           '**\"USD_VND\", \"USD_EUR\", \"EUR_VND\"**. Used to determine the exchange rate pair\n' + \
           'Each upload file will also need to contain all required features in the form of columns:\n' + \
           '\"Price\", \"Open\", \"High\", \"Low\". Optional column: \"Date\"'
    note_holder = gr.Markdown(note, line_breaks=True)

  with gr.Row(visible=False) as manual_note_row:
    note = '**INPUT NOTES:**\nThousands separator is not necessary or **COMMA** (,)\n' + \
           'Decimals separator is **DOT** (.)\n_Example:_ 23500.7580 or 23,500.7580'
    note_holder = gr.Markdown(note, line_breaks=True)

  with gr.Row(visible=False) as upload_row:
    uploads = gr.File(file_count='multiple')

  with gr.Row(visible=False) as manual_row:
    with gr.Column():
      with gr.Row() as df_row:
        with gr.Tab('Input Dataframe 1') as manual_tab1:
          with gr.Column():
            pair_dropdown1 = gr.Dropdown(choices=['USD-VND', 'USD-EUR', 'EUR-VND'],
                                         value='USD-VND', label="Select Exchange Pair")

            input_df1 = gr.Dataframe(height=300, headers=['Price', 'Open', 'High', 'Low'],
                                    col_count=[4,'fixed'], row_count=[8, 'dynamic'],
                                    interactive=True, column_widths=['600'*5])

        with gr.Tab('Input Dataframe 2') as manual_tab2:
          with gr.Column():
            pair_dropdown2 = gr.Dropdown(choices=['USD-VND', 'USD-EUR', 'EUR-VND'],
                                         value='USD-VND', label="Select Exchange Pair")

            input_df2 = gr.Dataframe(height=300, headers=['Price', 'Open', 'High', 'Low'],
                                    col_count=[4,'fixed'], row_count=[8, 'dynamic'],
                                    interactive=True, column_widths=['600'*5])

        with gr.Tab('Input Dataframe 3') as manual_tab3:
          with gr.Column():
            pair_dropdown3 = gr.Dropdown(choices=['USD-VND', 'USD-EUR', 'EUR-VND'],
                                         value='USD-VND', label="Select Exchange Pair")

            input_df3 = gr.Dataframe(height=300, headers=['Price', 'Open', 'High', 'Low'],
                                    col_count=[4,'fixed'], row_count=[8, 'dynamic'],
                                    interactive=True, column_widths=['600'*5])
      with gr.Row() as check_btn_row:
        check_btn = gr.Button('Check Format', interactive=True)

  with gr.Row() as execute_row:
    execute_btn = gr.Button("Execute Prediction", interactive = False)

  with gr.Row() as row_1:
    display_df1 = gr.DataFrame(interactive = False, visible = False)

  with gr.Row() as row_2:
    display_df2 = gr.DataFrame(interactive = False, visible = False)

  with gr.Row() as row_3:
    display_df3 = gr.DataFrame(interactive = False, visible = False)

  with gr.Row(visible=False) as row_4:
    with gr.Column():
      with gr.Row():
        markdown1 = gr.Markdown(visible = False)

      with gr.Row():
        with gr.Tab("Support Vector Regressor") as t1_1:
          fr1_1 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("K-Nearest Neighbors") as t1_2:
          fr1_2 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Random Forest") as t1_3:
          fr1_3 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("XGBoost") as t1_4:
          fr1_4 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Multilayer Perceptron") as t1_5:
          fr1_5 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])

  with gr.Row(visible=False) as row_5:
    with gr.Column():
      with gr.Row():
        markdown2 = gr.Markdown(visible = False)

      with gr.Row():
        with gr.Tab("Support Vector Regressor") as t2_1:
          fr2_1 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("K-Nearest Neighbors") as t2_2:
          fr2_2 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Random Forest") as t2_3:
          fr2_3 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("XGBoost") as t2_4:
          fr2_4 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Multilayer Perceptron") as t2_5:
          fr2_5 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])

  with gr.Row(visible=False) as row_6:
    with gr.Column():
      with gr.Row():
        markdown3 = gr.Markdown(visible = False)

      with gr.Row():
        with gr.Tab("Support Vector Regressor") as t3_1:
          fr3_1 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("K-Nearest Neighbors") as t3_2:
          fr3_2 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Random Forest") as t3_3:
          fr3_3 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("XGBoost") as t3_4:
          fr3_4 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])
        with gr.Tab("Multilayer Perceptron") as t3_5:
          fr3_5 = gr.DataFrame(interactive = False, visible = True, column_widths=['500'*6])

  radio.change(
      update_input_method,
      radio,
      [upload_row, manual_row, execute_btn, upload_note_row, manual_note_row]
  )

  input_df1.change(
      update_btn,
      gr.Textbox('Check Format1', visible=False),
      check_btn
  ).then(
      update_btn,
      gr.Textbox('Execute Prediction0', visible=False),
      execute_btn
  )

  input_df2.change(
      update_btn,
      gr.Textbox('Check Format1', visible=False),
      check_btn
  ).then(
      update_btn,
      gr.Textbox('Execute Prediction0', visible=False),
      execute_btn
  )

  input_df3.change(
      update_btn,
      gr.Textbox('Check Format1', visible=False),
      check_btn
  ).then(
      update_btn,
      gr.Textbox('Execute Prediction0', visible=False),
      execute_btn
  )

  uploads.upload(
          read_files,
          uploads,
          []) \
  .then(
          update_btn,
          gr.Textbox('Uploading...0', visible=False),
          execute_btn
          ) \
  .then(
          update_layout_invisible,
          value_3,
          [row_4, row_5, row_6]) \
  .then(
          update_layout_visible,
          value_3,
          [row_1, row_2, row_3, dummy]) \
  .then(
          update_display_visible,
          [gr.Textbox('initial display', visible=False)],
          [display_df1, display_df2, display_df3]) \
  .then(
          load_df,
          [],
          [display_df1, display_df2, display_df3, execute_btn]) \
  .then(
          update_btn,
          gr.Textbox('Execute prediction1', visible=False),
          execute_btn)

  uploads.clear(
          update_interactive_false,
          [],
          execute_btn)

  check_btn.click(
          update_layout_invisible,
          value_3,
          [row_1, row_2, row_3]) \
  .then(
          update_layout_invisible,
          value_3,
          [row_4, row_5, row_6]) \
  .then(
          check_manual_legit,
          [input_df1, input_df2, input_df3, pair_dropdown1, pair_dropdown2, pair_dropdown3],
          [check_btn, execute_btn]
  )

  execute_btn.click(
          check_no_file,
          [],
          []) \
  .then(
          update_btn,
          gr.Textbox('Executing Data...0', visible=False),
          execute_btn) \
  .then(
          update_layout_invisible,
          value_3,
          [row_1, row_2, row_3]) \
  .then(
          update_layout_visible,
          value_1,
          [row_4, dummy]) \
  .then(
          update_markdowns,
          [],
          [markdown1]) \
  .then(
          execute_prediction,
          [],
          [fr1_1, fr1_2, fr1_3, fr1_4, fr1_5]) \
  .then(
          update_layout_visible,
          value_1,
          [row_5, dummy]) \
  .then(
          update_markdowns,
          [],
          [markdown2]) \
  .then(
          execute_prediction,
          [],
          [fr2_1, fr2_2, fr2_3, fr2_4, fr2_5]) \
  .then(
          update_layout_visible,
          value_1,
          [row_6, dummy]) \
  .then(
          update_markdowns,
          [],
          [markdown3]) \
  .then(
          execute_prediction,
          [],
          [fr3_1, fr3_2, fr3_3, fr3_4, fr3_5]) \
  .then(
          update_btn,
          gr.Textbox('Prediction completed!0', visible=False),
          execute_btn)

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d1fb8aefc483d60720.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d1fb8aefc483d60720.gradio.live
